In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np

TensorFlow 1.x selected.


# lab-12-2. Hi Hello Training

In [ ]:
tf.set_random_seed(777)  # reproducibility

In [ ]:
input_dim = 5 # one-hot size  
hidden_size = 5  # output from the LSTM. 5 to directly predict one-hot
batch_size = 1   # one sentence
sequence_length = 6  # |ihello| == 6
num_classes = 5
learning_rate = 0.1

- Manual data creation

In [ ]:
# voc index (dic)
# h:0, i:1, e:2, l:3, o:4
idx2char = ['h', 'i', 'e', 'l', 'o'] 

x_data = [[0, 1, 0, 2, 3, 3]]   # hihell
x_one_hot = [[[1, 0, 0, 0, 0],   # h 0
              [0, 1, 0, 0, 0],   # i 1
              [1, 0, 0, 0, 0],   # h 0
              [0, 0, 1, 0, 0],   # e 2
              [0, 0, 0, 1, 0],   # l 3
              [0, 0, 0, 1, 0]]]  # l 3
              
y_data = [[1, 0, 2, 3, 3, 4]]    # ihello

In [ ]:
X = tf.placeholder(
    tf.float32, [None, sequence_length, input_dim])  # X one-hot
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label

In [ ]:
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True) # RNN, GRU 등 활용 가능
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(
    cell, X, initial_state=initial_state, dtype=tf.float32)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
# # FC layer
# X_for_fc = tf.reshape(outputs, [-1, hidden_size])
# # fc_w = tf.get_variable("fc_w", [hidden_size, num_classes])
# # fc_b = tf.get_variable("fc_b", [num_classes])
# # outputs = tf.matmul(X_for_fc, fc_w) + fc_b
# outputs = tf.contrib.layers.fully_connected(
#     inputs=X_for_fc, num_outputs=num_classes, activation_fn=None)

# # reshape out for sequence_loss
# outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

In [ ]:
# 시퀀스 내 각 요소들의 중요도, 가중치 -> loss 계산
weights = tf.ones([batch_size, sequence_length])  # 전부 1로 동일하게 설정

sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l, _ = sess.run([loss, train], feed_dict={X: x_one_hot, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_one_hot})
        print(i, "loss:", l, "prediction: ", result, "true Y: ", y_data)

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print("\tPrediction str: ", ''.join(result_str))


0 loss: 1.6107723 prediction:  [[2 3 3 4 4 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  ellooo
1 loss: 1.5258473 prediction:  [[2 3 3 3 4 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  ellloo
2 loss: 1.4532484 prediction:  [[2 3 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elello
3 loss: 1.3810188 prediction:  [[2 3 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elelll
4 loss: 1.3161987 prediction:  [[2 3 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elelll
5 loss: 1.2625374 prediction:  [[2 3 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elelll
6 loss: 1.2124075 prediction:  [[2 3 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elelll
7 loss: 1.1680177 prediction:  [[2 3 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elelll
8 loss: 1.1323599 prediction:  [[2 3 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  elelll
9 loss: 1.1045314 prediction:  [[2 3 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Pred

# lab12-3: Long Sequence RNN

In [ ]:
tf.reset_default_graph()
tf.set_random_seed(777)  # reproducibility

In [ ]:
sample = " if you want you"
idx2char = list(set(sample))  # index -> char, set() 이용하여 unique한 값만
idx2char

['a', ' ', 'i', 't', 'n', 'o', 'w', 'y', 'u', 'f']

In [ ]:
char2idx = {c: i for i, c in enumerate(idx2char)}  # char -> index
char2idx

{' ': 1,
 'a': 0,
 'f': 9,
 'i': 2,
 'n': 4,
 'o': 5,
 't': 3,
 'u': 8,
 'w': 6,
 'y': 7}

In [ ]:
sample_idx = [char2idx[c] for c in sample]  # char to index
sample_idx

[1, 2, 9, 1, 7, 5, 8, 1, 6, 0, 4, 3, 1, 7, 5, 8]

In [ ]:
# hyper parameters
dic_size = len(char2idx)  # RNN input size (one hot size)
hidden_size = len(char2idx)  # RNN output size
num_classes = len(char2idx)  # final output size (RNN or softmax, etc.)
batch_size = 1  # one sample data, one batch
sequence_length = len(sample) - 1  # number of lstm rollings (unit #)
learning_rate = 0.1

In [ ]:
x_data = [sample_idx[:-1]]  # X data sample (0 ~ n-1) hello: hell
y_data = [sample_idx[1:]]   # Y label sample (1 ~ n) hello: ello

X = tf.placeholder(tf.int32, [None, sequence_length])  # X data
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label

In [ ]:
x_one_hot = tf.one_hot(X, num_classes)  # one hot: 1 -> 0 1 0 0 0 0 0 0 0 0, dimension(shape)의 변화에 유의

In [ ]:
cell = tf.contrib.rnn.BasicLSTMCell(
    num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(
    cell, x_one_hot, initial_state=initial_state, dtype=tf.float32)

In [ ]:
weights = tf.ones([batch_size, sequence_length]) # 모두 1로 설정
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1000):
        l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_data})

        if i%200==0:
          # print char using dic
          result_str = [idx2char[c] for c in np.squeeze(result)]
          print(i, "loss:", l, "Prediction:", ''.join(result_str))

0 loss: 2.3081312 Prediction: o    u     u   
100 loss: 1.0553964 Prediction: if you want you
200 loss: 1.0463456 Prediction: if you want you
300 loss: 1.0453092 Prediction: if you want you
400 loss: 1.0448371 Prediction: if you want you
500 loss: 1.0445726 Prediction: if you want you
600 loss: 1.0444039 Prediction: if you want you
700 loss: 1.0442843 Prediction: if you want you
800 loss: 1.0442002 Prediction: if you want you
900 loss: 1.0441395 Prediction: if you want you


### Really long sentence

In [ ]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [ ]:
char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}

sequence_length = 10  # Any arbitrary number

In [ ]:
dataX = []
dataY = []
for i in range(0, len(sentence) - sequence_length): # 문장을 자르기, 윈도우 크기만큼
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    print(i, x_str, '->', y_str)

    x = [char_dic[c] for c in x_str]  # x str to index
    y = [char_dic[c] for c in y_str]  # y str to index

    dataX.append(x)
    dataY.append(y)

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [ ]:
data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)
batch_size = len(dataX) # 169개

learning_rate = 0.1

In [ ]:
X = tf.placeholder(tf.int32, [None, sequence_length])
Y = tf.placeholder(tf.int32, [None, sequence_length])

# One-hot encoding
X_one_hot = tf.one_hot(X, num_classes)
print(X_one_hot)  # check out the shape -> (batch, sequence_length, input_dim or num_classes)

Tensor("one_hot_1:0", shape=(?, 10, 25), dtype=float32)


In [ ]:
# Make a lstm cell with hidden_size (each unit output vector size)
def lstm_cell():
    cell = tf.contrib.rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
    return cell

multi_cells = tf.contrib.rnn.MultiRNNCell([lstm_cell() for _ in range(2)], state_is_tuple=True)

# outputs: unfolding size x hidden size, state = hidden size
outputs, _states = tf.nn.dynamic_rnn(multi_cells, X_one_hot, dtype=tf.float32)

In [ ]:
prediction = tf.argmax(outputs, axis=2)

# All weights are 1 (equal weights)
weights = tf.ones([batch_size, sequence_length])

sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
mean_loss = tf.reduce_mean(sequence_loss)
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(mean_loss)

In [ ]:
# without softmax -> doesn't work
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(500):
    _, l, results = sess.run(
        [train_op, mean_loss, outputs], feed_dict={X: dataX, Y: dataY})    
    if i%200==0:
      for j, result in enumerate(results):
          index = np.argmax(result, axis=1)
          print(i, j, ''.join([char_set[t] for t in index]), l)

# Let's print the last char of each result to check it works
results = sess.run(outputs, feed_dict={X: dataX})
for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j is 0:  # print all for the first result to make a sentence
        print(''.join([char_set[t] for t in index]), end='')
    else:
        print(char_set[index[-1]], end='')

0 0 grrrrbbbw, 3.2194784
0 1 bbb'bbbwdd 3.2194784
0 2 wwwtbwdddd 3.2194784
0 3 t'tttddddd 3.2194784
0 4 ',,ddddddd 3.2194784
0 5 aldddddddd 3.2194784
0 6 wddddddddw 3.2194784
0 7 dddddddddw 3.2194784
0 8 ,,,,ddwwww 3.2194784
0 9 ,rdwwwwwww 3.2194784
0 10 uwwwwwwwww 3.2194784
0 11 wwwwwwwwww 3.2194784
0 12 udwwwwooow 3.2194784
0 13 'wwwtbooww 3.2194784
0 14 wllooooo.w 3.2194784
0 15 ooooooo..w 3.2194784
0 16 amgmom www 3.2194784
0 17 ggggg wwgm 3.2194784
0 18 ggg  wwwmm 3.2194784
0 19 f   wwwds' 3.2194784
0 20 w,,wwwsss' 3.2194784
0 21 ,,.,ssss'' 3.2194784
0 22 wwwgs''''' 3.2194784
0 23 oogc'''''' 3.2194784
0 24 ggg''''''' 3.2194784
0 25 gh'''''''' 3.2194784
0 26 ,'''''''bb 3.2194784
0 27 '''''''rww 3.2194784
0 28 wwwww,bbwp 3.2194784
0 29 ffffnbbppp 3.2194784
0 30 '',rr,pppu 3.2194784
0 31 ,,rr,ppauu 3.2194784
0 32 ,,,lppaulo 3.2194784
0 33 uwwwwuuuoo 3.2194784
0 34 wwwwmuuooo 3.2194784
0 35 fmmmuooooo 3.2194784
0 36 mmmuuooaoo 3.2194784
0 37 aoolaaao.. 3.2194784
0 38 uulaaa...b 3.2194

# lab12-4: Stacked RNN + Softmax Layer

### with softmax layer

In [ ]:
tf.reset_default_graph()
tf.set_random_seed(777)  # reproducibility

In [ ]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}

data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)
sequence_length = 10  # Any arbitrary number
learning_rate = 0.1

dataX = []
dataY = []
for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    # print(i, x_str, '->', y_str)

    x = [char_dic[c] for c in x_str]  # x str to index
    y = [char_dic[c] for c in y_str]  # y str to index

    dataX.append(x)
    dataY.append(y)

batch_size = len(dataX)

X = tf.placeholder(tf.int32, [None, sequence_length])
Y = tf.placeholder(tf.int32, [None, sequence_length])

# One-hot encoding
X_one_hot = tf.one_hot(X, num_classes)
print(X_one_hot)  # check out the shape


# Make a lstm cell with hidden_size (each unit output vector size)
def lstm_cell():
    cell = tf.contrib.rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
    return cell

multi_cells = tf.contrib.rnn.MultiRNNCell([lstm_cell() for _ in range(5)], state_is_tuple=True)

# outputs: unfolding size x hidden size, state = hidden size
outputs, _states = tf.nn.dynamic_rnn(multi_cells, X_one_hot, dtype=tf.float32)

Tensor("one_hot:0", shape=(?, 10, 25), dtype=float32)


In [ ]:
# softmax layer (hidden_size -> num_classes)
X_for_softmax = tf.reshape(outputs, [-1, hidden_size])
softmax_w = tf.get_variable("softmax_w", [hidden_size, num_classes], initializer = tf.contrib.layers.variance_scaling_initializer()) 
# tf.contrib.layers.he_initializer())
softmax_b = tf.get_variable("softmax_b", [num_classes])
outputs = tf.matmul(X_for_softmax, softmax_w) + softmax_b

# expend the data (revive the batches)
# reshape out for sequence_Loss (after softmax)
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])
# All weights are 1 (equal weights)
weights = tf.ones([batch_size, sequence_length])

# prediction = tf.argmax(outputs, axis=2)
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)
mean_loss = tf.reduce_mean(sequence_loss)
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(mean_loss)     

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(1001):
    _, l, results = sess.run(
        [train_op, mean_loss, outputs], feed_dict={X: dataX, Y: dataY})
    print(i, '', l)

    # if i%1000==0:
    #   for j, result in enumerate(results):
    #       index = np.argmax(result, axis=1)
    #       print(i, j, ''.join([char_set[t] for t in index]), l)

# Let's print the last char of each result to check it works
results = sess.run(outputs, feed_dict={X: dataX})
for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j is 0:  # print all for the first result to make a sentence
        print(''.join([char_set[t] for t in index]), end='')
    else:
        print(char_set[index[-1]], end='')

0  3.2344236
1  3.120576
2  3.069551
3  2.998508
4  2.9271743
5  2.893903
6  2.8721611
7  2.8567371
8  2.8487723
9  2.8473954
10  2.8472323
11  2.8457842
12  2.8440576
13  2.842527
14  2.8418858
15  2.8406425
16  2.84007
17  2.8393471
18  2.8386722
19  2.838345
20  2.8376846
21  2.8373775
22  2.8371375
23  2.8363724
24  2.8351016
25  2.832251
26  2.828016
27  2.8183424
28  2.8026826
29  2.778915
30  2.746993
31  2.7096167
32  2.672673
33  2.7086205
34  2.6492112
35  2.7028637
36  2.5903518
37  2.6374779
38  2.5735476
39  2.526565
40  2.5079806
41  2.497406
42  2.4550471
43  2.4120102
44  2.3872325
45  2.369813
46  2.3300629
47  2.3139083
48  2.282913
49  2.245112
50  2.2156663
51  2.1851528
52  2.1496077
53  2.1216025
54  2.0763943
55  2.0330324
56  2.1253486
57  2.055747
58  2.0208194
59  1.9579037
60  1.9424263
61  1.8922606
62  1.8861575
63  1.8165474
64  1.7822266
65  1.8805727
66  1.864057
67  1.746391
68  1.742532
69  1.727791
70  1.6485314
71  1.656281
72  1.6485142
73  1.584441

### with FC layer

In [ ]:
tf.reset_default_graph()
tf.set_random_seed(777)  # reproducibility

In [ ]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}

data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)
sequence_length = 10  # Any arbitrary number
learning_rate = 0.1

dataX = []
dataY = []
for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    # print(i, x_str, '->', y_str)

    x = [char_dic[c] for c in x_str]  # x str to index
    y = [char_dic[c] for c in y_str]  # y str to index

    dataX.append(x)
    dataY.append(y)

batch_size = len(dataX)

X = tf.placeholder(tf.int32, [None, sequence_length])
Y = tf.placeholder(tf.int32, [None, sequence_length])

# One-hot encoding
X_one_hot = tf.one_hot(X, num_classes)
print(X_one_hot)  # check out the shape


# Make a lstm cell with hidden_size (each unit output vector size)
def lstm_cell():
    cell = tf.contrib.rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
    return cell

multi_cells = tf.contrib.rnn.MultiRNNCell([lstm_cell() for _ in range(2)], state_is_tuple=True)

# outputs: unfolding size x hidden size, state = hidden size
outputs, _states = tf.nn.dynamic_rnn(multi_cells, X_one_hot, dtype=tf.float32)

Tensor("one_hot:0", shape=(?, 10, 25), dtype=float32)


In [ ]:
# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

# reshape out for sequence_loss (after softmax)
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

# All weights are 1 (equal weights)
weights = tf.ones([batch_size, sequence_length])

sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
mean_loss = tf.reduce_mean(sequence_loss)
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(mean_loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(500):
    _, l, results = sess.run(
        [train_op, mean_loss, outputs], feed_dict={X: dataX, Y: dataY})
    for j, result in enumerate(results):
        index = np.argmax(result, axis=1)
        print(i, j, ''.join([char_set[t] for t in index]), l)

# Let's print the last char of each result to check it works
results = sess.run(outputs, feed_dict={X: dataX})
for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j is 0:  # print all for the first result to make a sentence
        print(''.join([char_set[t] for t in index]), end='')
    else:
        print(char_set[index[-1]], end='')

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
470 101 iand work, 0.22926114
470 102 tnd dork,  0.22926114
470 103 nd work, b 0.22926114
470 104 d dork, bu 0.22926114
470 105  aork, but 0.22926114
470 106 tork, but  0.22926114
470 107 ork, but r 0.22926114
470 108 nk, but ra 0.22926114
470 109  , but rat 0.22926114
470 110 s but rath 0.22926114
470 111  dut rathe 0.22926114
470 112 tut rather 0.22926114
470 113 ui rather  0.22926114
470 114 t rather t 0.22926114
470 115 haather te 0.22926114
470 116 tather tea 0.22926114
470 117  ther teac 0.22926114
470 118 nher teach 0.22926114
470 119 hem toach  0.22926114
470 120 em teach t 0.22926114
470 121 n toach th 0.22926114
470 122  thach the 0.22926114
470 123 toach them 0.22926114
470 124 hach them  0.22926114
470 125 nch them t 0.22926114
470 126 nh them to 0.22926114
470 127 t them to  0.22926114
470 128 ethem to l 0.22926114
470 129 toem ta lo 0.22926114
470 130 hem ta lon 0.22926114
470 131 em ta long 0.22926114
470 132 n ta long  0.22926114
470 